<a href="https://colab.research.google.com/github/Arbaj-Wadagera/AI_LAB-2024-25/blob/main/1BM22CS051_forward_reasoning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
class ForwardChainingFOL:
    def __init__(self):
        self.facts = set()
        self.rules = []

    def add_fact(self, fact):
        """Add a fact to the knowledge base."""
        self.facts.add(fact)

    def add_rule(self, premises, conclusion):
        """Add a rule to the knowledge base."""
        self.rules.append((premises, conclusion))

    def unify(self, fact, premise):
        """Attempt to unify a fact with a rule's premise."""
        if "(" not in premise:
            return fact == premise
        if premise.split("(")[0] != fact.split("(")[0]:
            return None

        fact_args = fact[fact.index("(") + 1: -1].split(",")
        premise_args = premise[premise.index("(") + 1: -1].split(",")
        if len(fact_args) != len(premise_args):
            return None

        substitution = {}
        for f_arg, p_arg in zip(fact_args, premise_args):
            if p_arg.islower():
                if p_arg in substitution:
                    if substitution[p_arg] != f_arg:
                        return None
                else:
                    substitution[p_arg] = f_arg
            elif f_arg != p_arg:
                return None
        return substitution

    def apply_substitution(self, expr, substitution):
        """Apply a substitution to an expression."""
        if "(" not in expr:
            return substitution.get(expr, expr)

        predicate = expr.split("(")[0]
        args = expr[expr.index("(") + 1: -1].split(",")
        substituted_args = [substitution.get(arg, arg) for arg in args]
        return f"{predicate}({','.join(substituted_args)})"

    def infer(self):
        """Perform forward chaining to infer all possible facts."""
        new_inferences = True
        while new_inferences:
            new_inferences = False
            for premises, conclusion in self.rules:
                substitutions = []
                for premise in premises:
                    for fact in self.facts:
                        subst = self.unify(fact, premise)
                        if subst is not None:
                            substitutions.append(subst)
                            break

                if len(substitutions) == len(premises):
                    combined_subst = {}
                    for subst in substitutions:
                        combined_subst.update(subst)

                    inferred_fact = self.apply_substitution(conclusion, combined_subst)
                    if inferred_fact not in self.facts:
                        self.format_inference_output(inferred_fact)
                        self.facts.add(inferred_fact)
                        new_inferences = True

    def format_inference_output(self, inferred_fact):
        """Format the output for inferred facts."""
        print(f"--- Inference Made ---")
        print(f"New Fact Inferred: {inferred_fact}")
        print(f"Current Knowledge Base: {self.facts}")
        print("-----------------------\n")

    def query(self, query_fact):
        """Query if a specific fact can be inferred."""
        if query_fact in self.facts:
            self.format_query_output(query_fact, True)
            return True
        else:
            self.format_query_output(query_fact, False)
            return False

    def format_query_output(self, query_fact, is_proved):
        """Format the output for query results."""
        if is_proved:
            print(f"--- Query Result ---")
            print(f"Fact Proven: {query_fact}")
        else:
            print(f"--- Query Result ---")
            print(f"Fact Could Not Be Proven: {query_fact}")
        print("-----------------------\n")

# Example usage
kb = ForwardChainingFOL()

kb.add_fact("American(Robert)")
kb.add_fact("Missile(T1)")
kb.add_fact("Owns(A,T1)")
kb.add_fact("Enemy(A,America)")

kb.add_rule(["Missile(x)", "Owns(A,x)"], "Sells(Robert,x,A)")
kb.add_rule(["Missile(x)"], "Weapon(x)")
kb.add_rule(["Enemy(x,America)"], "Hostile(x)")
kb.add_rule(["American(p)", "Weapon(q)", "Sells(p,q,r)", "Hostile(r)"], "Criminal(p)")

kb.infer()

kb.query("Criminal(Robert)")

--- Inference Made ---
New Fact Inferred: Sells(Robert,T1,A)
Current Knowledge Base: {'Owns(A,T1)', 'American(Robert)', 'Missile(T1)', 'Enemy(A,America)'}
-----------------------

--- Inference Made ---
New Fact Inferred: Weapon(T1)
Current Knowledge Base: {'Enemy(A,America)', 'Owns(A,T1)', 'Missile(T1)', 'Sells(Robert,T1,A)', 'American(Robert)'}
-----------------------

--- Inference Made ---
New Fact Inferred: Hostile(A)
Current Knowledge Base: {'Enemy(A,America)', 'Weapon(T1)', 'Owns(A,T1)', 'Missile(T1)', 'Sells(Robert,T1,A)', 'American(Robert)'}
-----------------------

--- Inference Made ---
New Fact Inferred: Criminal(Robert)
Current Knowledge Base: {'Enemy(A,America)', 'Weapon(T1)', 'Owns(A,T1)', 'Missile(T1)', 'Sells(Robert,T1,A)', 'American(Robert)', 'Hostile(A)'}
-----------------------

--- Query Result ---
Fact Proven: Criminal(Robert)
-----------------------



True